In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from  sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from statsmodels.tsa.seasonal import seasonal_decompose

label_encoder = LabelEncoder()


InputLayer = keras.layers.InputLayer()
LSTM = keras.layers.LSTM
Dropout = keras.layers.Dropout
Dense = keras.layers.Dense
skencoder = OneHotEncoder(handle_unknown='ignore',sparse=False)


ModuleNotFoundError: No module named 'tensorflow'

In [118]:
df = pd.read_csv('/Users/victoroliveira/Desktop/vonix-py-statistic/src/vonixstatisc/data/pandas_data/data_fluency_queue_no_auto_dialer.csv')

df_to_dict = pd.DataFrame()
df= df.dropna(axis=0)
del df['direction'], df['locality_id'], df['call_type_id'], df['hold_secs'], df['ring_secs'], df['initial_position'] , df['carrier_id'], df['timestamp'], df['day'], df['month'], df['trunking_id'] 

In [119]:
df_to_ml = pd.get_dummies(df, prefix=None, dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)
df_to_ml
#sk_data = skencoder.fit_transform(df)
#sk_data, sk_data.shape
#skencoder.get_feature_names_out()

,agent_id,week_day,hour,minute,handling_time,queue_id_equipez4,queue_id_equipez5,queue_id_equipez6,queue_id_isdragons,queue_id_iseagles,...,queue_id_isnnadal,queue_id_isnrecuperacao,queue_id_isnxsell,queue_id_isphoenix,queue_id_issharks,queue_id_istigers,queue_id_iswolves,queue_id_mgmsales,queue_id_mgmsdr,queue_id_timec
0,1000,4,20,2,78,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000,4,21,2,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000,4,21,2,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000,4,21,2,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000,4,21,2,23,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117583,1573,1,21,4,47,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
117584,1137,1,21,4,70,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117585,1572,1,21,4,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
117586,1600,1,21,4,4,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [104]:
df_to_ml.iloc[:, 0:].values

array([[1000,    4,   20, ...,    0,    0,    0],
       [1000,    4,   21, ...,    0,    0,    0],
       [1000,    4,   21, ...,    0,    0,    0],
       ...,
       [1572,    1,   21, ...,    0,    0,    0],
       [1600,    1,   21, ...,    1,    0,    0],
       [1572,    1,   21, ...,    0,    0,    0]])

In [105]:
normalizador = MinMaxScaler(feature_range = (0,1))
base_treinamento_normalizada = normalizador.fit_transform(df_to_ml)

In [106]:
base_treinamento_normalizada.shape

(115277, 30)

In [107]:
previsores = []
tempo_real = []

In [108]:
for i in range(1, 115277):
    previsores.append(base_treinamento_normalizada[i-1:i, 0:30])
    tempo_real.append(base_treinamento_normalizada[i,4])

In [109]:
#previsores

In [110]:
previsores, tempo_real = np.array(previsores), np.array(tempo_real)
previsores= np.reshape(previsores, (previsores.shape[0], previsores.shape[1],30))

In [111]:
previsores.shape[0:]

(115276, 1, 30)

In [112]:
#pd.DataFrame(tempo_real)

In [120]:
regressor = keras.models.Sequential()
#camadas inicial
regressor.add(LSTM(units = 100, return_sequences = True, input_shape=(1,30)))
regressor.add(Dropout(0.3))     

#camadas
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))    

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))  

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3)) 

#camadas finais
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3)) 

regressor.add(Dense(units = 1, activation = 'sigmoid'))

regressor.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



In [121]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [122]:
es = EarlyStopping(monitor = 'loss', min_delta = 1e-10, patience = 10 , verbose =1)
rlr = ReduceLROnPlateau(monitor = 'loss', factoe = 0.2, patience = 5, verbose = 1)
mcp = ModelCheckpoint(filepath = 'pesos.h5', monitor = 'loss', save_best_only = True)

In [124]:
regressor.fit(previsores, tempo_real, epochs = 10, batch_size = 3)

Epoch 1/10
38426/38426 [==============================] - 64s 2ms/step - loss: 0.0941 - accuracy: 0.1448
Epoch 2/10
38426/38426 [==============================] - 64s 2ms/step - loss: 0.0937 - accuracy: 0.1448
Epoch 3/10
38426/38426 [==============================] - 64s 2ms/step - loss: 0.0935 - accuracy: 0.1448
Epoch 4/10
38426/38426 [==============================] - 66s 2ms/step - loss: 0.0934 - accuracy: 0.1448
Epoch 5/10
38426/38426 [==============================] - 66s 2ms/step - loss: 0.0933 - accuracy: 0.1448
Epoch 6/10
38426/38426 [==============================] - 66s 2ms/step - loss: 0.0933 - accuracy: 0.1448
Epoch 7/10
38426/38426 [==============================] - 64s 2ms/step - loss: 0.0932 - accuracy: 0.1448
Epoch 8/10
38426/38426 [==============================] - 64s 2ms/step - loss: 0.0932 - accuracy: 0.1448
Epoch 9/10
38426/38426 [==============================] - 64s 2ms/step - loss: 0.0932 - accuracy: 0.1448
Epoch 10/10
38426/38426 [==============================